# 2. Preprocessing
We will address every single point that is mentioned in the Preprocessing slides. We won't have to code everyone but at least state why we are doing it or why not. We are addressing the following:

Errors, Missing Values, Unbalanced distribution, Different Scales, False Predictors, Unsupported data types (Categorical data and Dates, Textual values), High Dimensionality (Feature Subset Selection, PCA, Sampling)

## Packages to use
Make sure, that we use the packages that were referenced in the slides (Preprocessing, slide 70):
- scikit-learn Imputation
- scikit-learn Preprocessing
- scikit-learn Text feature extraction
- scikit-learn Feature Selection

In [ ]:
import sys, site, platform, pandas as pd
from sklearn.preprocessing import OneHotEncoder

print(sys.executable)
print(platform.python_version())
print(pd.__version__)

c:\Users\komit\Documents\Education\University\University of Mannheim\1st Sem\Data Mining (IE500)\3. Group Project\datamining_group12\venv\Scripts\python.exe
3.11.1
2.3.3


In [2]:
import os
from utils_io import load_step, save_step

# Load the dataset
print(os.getcwd())
df = load_step("step1_selection")

c:\Users\komit\Documents\Education\University\University of Mannheim\1st Sem\Data Mining (IE500)\3. Group Project\datamining_group12


In [3]:
print(df.head(5))

                 track_id  popularity  duration_ms  explicit  danceability  \
0  5SuOikwiRyPMVoIQDJUgSV          73       230666     False         0.676   
1  4qPNDBW1i3p13qLCt0Ki3A          55       149610     False         0.420   
2  1iJBSr7s7jYXzM8EGcbK5b          57       210826     False         0.438   
3  6lfxq3CG4xtTiEg7opyCyx          71       201933     False         0.266   
4  5vjLSffimiIP26QG5WcN2K          82       198853     False         0.618   

   energy  key  loudness  mode  speechiness  acousticness  instrumentalness  \
0  0.4610    1    -6.746     0       0.1430        0.0322          0.000001   
1  0.1660    1   -17.235     1       0.0763        0.9240          0.000006   
2  0.3590    0    -9.734     1       0.0557        0.2100          0.000000   
3  0.0596    0   -18.515     1       0.0363        0.9050          0.000071   
4  0.4430    2    -9.681     1       0.0526        0.4690          0.000000   

   liveness  valence    tempo  time_signature track_genr

### Errors in data
Simple remedy, Anomaly Detection

### Missing Values

In [3]:
# Remove rows with Missing Values
df = df.dropna()

# # Verify that missing values are removed
# print('Dataset Info without Missing Values:')
# df.info()

### Unsupported Data Types
Transform textual attributes to vector representations.

In our case: track_genre is a nominal attribute. We are doing multi-label one hot encoding (Ask: Does this work because it is not one hot but multiple columns up to a maximum of 6 genres)

Key, Mode, Explicit, Time_signature are "textual" features, too. At least their meaning is textual and they represent therefore a label. Hence we will one-hot encode them. Mode and Explicit do not need to be encoded since they are binary. Whereas Key and Time_signature need to be encoded.

#### - Genre summarization

In [5]:
# keep original_track_genre
# add new superior_track_genre

# dictionary with key as keywords for super genre mapping
# 15 superior genres -> list of source tags
category_keywords = {
    "Pop": [
        "pop","power-pop","synth-pop","indie-pop","k-pop","j-pop","mandopop","cantopop","j-idol"
    ],
    "Rock": [
        "rock","rock-n-roll","rockabilly","alt-rock","alternative","indie","emo","grunge",
        "psych-rock","punk","punk-rock","hard-rock","goth","garage","j-rock"
    ],
    "Hip-Hop/Rap": [
        "hip-hop"
    ],
    "Electronic/Dance": [
        "electronic","edm","electro","dance","club","house","deep-house","progressive-house",
        "chicago-house","techno","detroit-techno","minimal-techno","trance","hardstyle",
        "drum-and-bass","breakbeat","dubstep","idm","j-dance","ambient","trip-hop","industrial"
    ],
    "Metal": [
        "metal","heavy-metal","black-metal","death-metal","metalcore","grindcore","hardcore"
    ],
    "Latin": [
        "latin","latino","reggaeton","salsa","samba","tango","pagode","forro","mpb","sertanejo"
    ],
    "R&B/Soul/Funk": [
        "r-n-b","soul","funk","gospel","groove"
    ],
    "Jazz/Blues": [
        "jazz","blues"
    ],
    "Country/Americana": [
        "country","honky-tonk","bluegrass"
    ],
    "Folk/Acoustic/Singer-Songwriter": [
        "folk","acoustic","singer-songwriter","songwriter","guitar"
    ],
    "Classical/Opera": [
        "classical","opera","piano"
    ],
    "Reggae/Ska/Dub": [
        "reggae","ska","dub","dancehall"
    ],
    "Soundtrack/Showtunes": [
        "show-tunes","disney","anime","pop-film"
    ],
    "World/International": [
        "world-music","brazil","french","german","spanish","swedish","turkish","malay",
        "iranian","indian","british"
    ],
    "Mood/Functional/Other": [
        "chill","happy","party","romance","sad","sleep","study","children","kids","new-age","comedy"
    ],
}

# Invert to tag -> super-genre map
super_genre_map = {
    tag: cat
    for cat, tags in category_keywords.items()
    for tag in tags
}


#### - Remove Duplicates

In [ ]:
# Find duplicates in df[track_id]
print("Duplicates track_id", sum(df["track_id"].duplicated()))
print("Duplicates whole dataframe", sum(df.duplicated()))

print("Duplicates track_id & track_genre", sum(df[["track_id", "track_genre"]].duplicated()))

# Do this for all combinations to show, that these are real duplicates and not (superficial duplicates as with the track_genre)
print("Duplicates track_id & track_genre & 3rd", sum(df[["track_id", "track_genre", "time_signature"]].duplicated()))

# --> 24259 duplicates in "track_id", 450 in whole dataset, combination of track_id and track_genre again 450
# --> Consequently, 450 true duplicates, rest alternations of track_genre

# Find all superior genres for one track_id (a df with columns track_id and superior_track_genre)
print("Unique genres:", df["track_genre"].unique())
print("Number of unique genres: ", len(df["track_genre"].unique()))


Duplicates track_id 24126
Duplicates whole dataframe 432
Duplicates track_id & track_genre 432
Duplicates track_id & track_genre & 3rd 432
Unique genres: ['acoustic' 'afrobeat' 'alt-rock' 'alternative' 'ambient' 'anime'
 'black-metal' 'bluegrass' 'blues' 'brazil' 'breakbeat' 'british'
 'cantopop' 'chicago-house' 'children' 'chill' 'classical' 'club' 'comedy'
 'country' 'dance' 'dancehall' 'death-metal' 'deep-house' 'detroit-techno'
 'disco' 'disney' 'drum-and-bass' 'dub' 'dubstep' 'edm' 'electro'
 'electronic' 'emo' 'folk' 'forro' 'french' 'funk' 'garage' 'german'
 'gospel' 'goth' 'grindcore' 'groove' 'grunge' 'guitar' 'happy'
 'hard-rock' 'hardcore' 'hardstyle' 'heavy-metal' 'hip-hop' 'honky-tonk'
 'house' 'idm' 'indian' 'indie-pop' 'indie' 'industrial' 'iranian'
 'j-dance' 'j-idol' 'j-pop' 'j-rock' 'jazz' 'k-pop' 'kids' 'latin'
 'latino' 'malay' 'mandopop' 'metal' 'metalcore' 'minimal-techno' 'mpb'
 'new-age' 'opera' 'pagode' 'party' 'piano' 'pop-film' 'pop' 'power-pop'
 'progressive

In [7]:
# df with track_id and track_genre
df_track_id_genre = df[["track_id", "track_genre"]]

# df with track_id, track_genre and genre_super
# Apply (single-tag per row). For unknown tags fall back to World/International or Other.
df_super_long = df_track_id_genre
df_super_long["genre_super"] = df_track_id_genre["track_genre"].map(super_genre_map).fillna("World/International")

# df with track_id and genre_super
df_super_long = df_super_long.drop(columns=["track_genre"])
print(df_super_long.head(5))


                 track_id                      genre_super
0  5SuOikwiRyPMVoIQDJUgSV  Folk/Acoustic/Singer-Songwriter
1  4qPNDBW1i3p13qLCt0Ki3A  Folk/Acoustic/Singer-Songwriter
2  1iJBSr7s7jYXzM8EGcbK5b  Folk/Acoustic/Singer-Songwriter
3  6lfxq3CG4xtTiEg7opyCyx  Folk/Acoustic/Singer-Songwriter
4  5vjLSffimiIP26QG5WcN2K  Folk/Acoustic/Singer-Songwriter


/tmp/ipykernel_3473/2712052728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_super_long["genre_super"] = df_track_id_genre["track_genre"].map(super_genre_map).fillna("World/International")


#### - Multi-Hot Encoding for Track Genre

We decided to not use the OneHotEncoder from the sklearn.preprocessing package since our data is multi-label. OneHotEncoder expects one label per category but we have multiple labels per category (aka track_id). So we would have to explode the genres and then aggregate anyway. Additionally our dataframe is static and we are not building a full ML Pipeline for automated retraining. Hence, we stick with the manual solution with pandas. Regardless the technique, we will have a dataframe with genres as columns and binary values indicating to which genres a song belongs.

In [8]:
# remove exact duplicates
df_super_long = df_super_long.drop_duplicates()

# create the one hot encoding for multi-label as a wide dataframe
df_wide = (
    df_super_long.assign(val=1)
    .pivot_table(index="track_id", columns="genre_super", values="val",
                 aggfunc="max", fill_value=0)
    .astype("int8")
)

df_wide.columns = [f"genre__{c.replace(' ', '_').replace('/', '_')}" for c in df_wide.columns]
df_wide.index.name = "track_id"  # ensure the index has the right name
df_wide = df_wide.reset_index()  # now it becomes a proper column

print(df_wide.head(5))

                 track_id  genre__Classical_Opera  genre__Country_Americana  \
0  0000vdREvCVMxbQTkS888c                       0                         0   
1  000CC8EParg64OmTxVnZ0p                       0                         0   
2  000Iz0K615UepwSJ5z2RE5                       0                         0   
3  000RDCYioLteXcutOjeweY                       0                         0   
4  000qpdoc97IMTBvF8gwcpy                       0                         0   

   genre__Electronic_Dance  genre__Folk_Acoustic_Singer-Songwriter  \
0                        0                                       0   
1                        1                                       0   
2                        1                                       0   
3                        0                                       0   
4                        1                                       0   

   genre__Hip-Hop_Rap  genre__Jazz_Blues  genre__Latin  genre__Metal  \
0                   0           

In [9]:
# join back to original dataframe df
base = df.drop(columns=["track_genre"]).copy()
base = base.drop_duplicates(subset=["track_id"])

# quick sanity check before merging
print("Length base:", len(base))
print("Length df_wide:", len(df_wide))

# merge with how=left to guarantee all base rows will be kept
df_final = base.merge(df_wide, on="track_id", how="left")

print("Length final dataset:", len(df_final))

Length base: 88266
Length df_wide: 88266
Length final dataset: 88266


In [11]:
# Quick Sanity checks
# 1) Uniqueness before merge (protects against row explosion)
assert base["track_id"].is_unique, "base has duplicate track_id"
assert df_wide["track_id"].is_unique, "df_wide has duplicate track_id"

# 2) No unexpected NaNs in the multi-hot columns after merge
genre_cols = [c for c in df_final.columns if c.startswith("genre__")]
na_counts = df_final[genre_cols].isna().sum().sum()
print("NaNs in genre columns:", na_counts)

# If any NaNs (shouldn’t happen if pivot used fill_value=0), fix:
# df_final[genre_cols] = df_final[genre_cols].fillna(0).astype("int8")

# 3) All values are 0/1
bad_vals = set(df_final[genre_cols].stack().unique()) - {0, 1}
print("Non {0,1} values present?:", bool(bad_vals), bad_vals)

# 4) No track has all-zero vector (would indicate unmapped genre)
all_zero_tracks = (df_final[genre_cols].sum(axis=1) == 0).sum()
print("Tracks with no super-genre assigned:", all_zero_tracks)

df_final["amount_genres"] = df_final[genre_cols].sum(axis=1)
print("Maximum of genres a song belonged:", max(df_final["amount_genres"]))

# 5) Basic distribution & sparsity (quick glance)
col_sums = df_final[genre_cols].sum().sort_values(ascending=False)
print(col_sums.head(10))

# 6) Optional: memory/dtype tidy-up
print(df_final[genre_cols].dtypes.unique())
# If needed:
# df_final[genre_cols] = df_final[genre_cols].astype("int8")


NaNs in genre columns: 0
Non {0,1} values present?: False set()
Tracks with no super-genre assigned: 0
Maximum of genres a song belonged: 6
genre__Electronic_Dance         18863
genre__World_International      12706
genre__Rock                     11957
genre__Mood_Functional_Other    10157
genre__Pop                       8714
genre__Latin                     8004
genre__Metal                     6343
genre__R&B_Soul_Funk             4668
genre__Reggae_Ska_Dub            3943
genre__Soundtrack_Showtunes      3788
dtype: int64
[dtype('int8')]


In [12]:
# set df_final to df
df = df_final
df.head(3)

,track_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,...,genre__Latin,genre__Metal,genre__Mood_Functional_Other,genre__Pop,genre__R&B_Soul_Funk,genre__Reggae_Ska_Dub,genre__Rock,genre__Soundtrack_Showtunes,genre__World_International,amount_genres
0,5SuOikwiRyPMVoIQDJUgSV,73,230666,False,0.676,0.461,1,-6.746,0,0.1430,...,0,0,0,1,0,0,0,0,0,2
1,4qPNDBW1i3p13qLCt0Ki3A,55,149610,False,0.420,0.166,1,-17.235,1,0.0763,...,0,0,1,0,0,0,0,0,0,2
2,1iJBSr7s7jYXzM8EGcbK5b,57,210826,False,0.438,0.359,0,-9.734,1,0.0557,...,0,0,0,0,0,0,0,0,0,1


#### - One-Hot Encoding for Key and Time Signature
Since trees do not use distances we will stick with OHE. Alternatively, we could have used cyclic encoding (with sin and cos). With that we would get a compact representation that respects musical adjacency. This might be useful if we later test linear regression, ridge/lasso, SVM, k-NN or neural nets. For our current scope One Hot Encoding is the optimal choice.

In [14]:
# use OHE from scikit package for key and time signature
# OneHotEncoder: fit encoder on the two categorical columns (later test against difference for 'infrequent_if_exist)
enc = OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=int)
encoded = enc.fit_transform(df[["key", "time_signature"]])

# Build a DataFrame with readable column names
ohe_cols = enc.get_feature_names_out(["key", "time_signature"])
encoded_df = pd.DataFrame(encoded, columns=ohe_cols, index=df.index)

# Replace the original columns with their OHE versions
df_simple = pd.concat([df.drop(columns=["key", "time_signature"]), encoded_df], axis=1)

df_simple.head()
 

,track_id,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,...,key_6,key_7,key_8,key_9,key_10,key_11,time_signature_1,time_signature_3,time_signature_4,time_signature_5
0,5SuOikwiRyPMVoIQDJUgSV,73,230666,False,0.676,0.4610,-6.746,0,0.1430,0.0322,...,0,0,0,0,0,0,0,0,1,0
1,4qPNDBW1i3p13qLCt0Ki3A,55,149610,False,0.420,0.1660,-17.235,1,0.0763,0.9240,...,0,0,0,0,0,0,0,0,1,0
2,1iJBSr7s7jYXzM8EGcbK5b,57,210826,False,0.438,0.3590,-9.734,1,0.0557,0.2100,...,0,0,0,0,0,0,0,0,1,0
3,6lfxq3CG4xtTiEg7opyCyx,71,201933,False,0.266,0.0596,-18.515,1,0.0363,0.9050,...,0,0,0,0,0,0,0,1,0,0
4,5vjLSffimiIP26QG5WcN2K,82,198853,False,0.618,0.4430,-9.681,1,0.0526,0.4690,...,0,0,0,0,0,0,0,0,1,0


### Unbalanced Class Distribution
Our outcome variable is continuous, so there is not really a class imbalance. Most of our predictor variables are continuous, too, however we do use genres which might be distributed inequally.

There are way more tracks labeled as rock, pop, world_international, latin, electronic_dance, mood_functional_other. 

In [15]:
# columns
features = ["popularity","danceability","energy","loudness","speechiness", "duration_ms"]
genre_cols = [c for c in df.columns if c.startswith("genre_")]  # or pass your explicit list

# long format → filter rows where the genre flag == 1 → mean by genre
long = df[features + genre_cols].melt(
    id_vars=features, value_vars=genre_cols,
    var_name="genre", value_name="flag"
)
summary = (long[long["flag"] == 1]
           .groupby("genre")[features]
           .mean()
           .sort_values("popularity", ascending=False))

# add counts per genre and share of dataset
counts = df[genre_cols].sum().rename("n_tracks")
summary = summary.join(counts, on="genre")
summary["share_%"] = 100 * summary["n_tracks"] / len(df)

summary.reset_index()


,genre,popularity,danceability,energy,loudness,speechiness,duration_ms,n_tracks,share_%
0,genre__Soundtrack_Showtunes,42.360876,0.520307,0.504339,-10.125147,0.082596,218471.058606,3788,4.291573
1,genre__Pop,39.115446,0.575121,0.655173,-7.081615,0.060949,230140.993803,8714,9.872431
2,genre__Hip-Hop_Rap,37.997976,0.736080,0.682104,-5.995789,0.129932,206708.891700,988,1.119344
3,genre__Rock,37.947980,0.527456,0.695715,-7.110491,0.064906,216717.267709,11957,13.546552
4,genre__Folk_Acoustic_Singer-Songwriter,37.454989,0.555113,0.434556,-10.502343,0.051500,219533.253254,3688,4.178279
5,genre__World_International,35.280104,0.567709,0.607267,-8.811255,0.081263,240496.964741,12706,14.395124
6,genre__R&B_Soul_Funk,35.199443,0.594690,0.634043,-7.083354,0.084511,237267.723008,4668,5.288560
7,genre__Latin,33.708521,0.625056,0.664559,-7.006199,0.074363,216923.118191,8004,9.068044
8,genre__Reggae_Ska_Dub,32.489221,0.658578,0.730774,-5.925630,0.116990,215814.770226,3943,4.467179
9,genre__Metal,31.367649,0.405089,0.883405,-5.678939,0.104453,234501.019234,6343,7.186233


### Different Scales: Normalization
Since we are computing a Regression Tree we do not need to scale the values between 0 and 1. We could implement it to guarantee consistency across models (e.g. comparing with model trees). In a model tree we would need to scale the values so the penalty in Lasso or Ridge regression stays fairly. However, so far we are asked to compute a model which interpolates and not one that extrapolates as model trees would we skip scaling.

### Multiple Attributes

### Feature Subset Selection

### Sampling
Our training data is already a sample.
Maybe stratified sampling by genre to ensure all genres are proportionally represented in each fold. Avoids overfitting to popular genres. (Do we really need this? Since we are doing a Regression Tree we split at genre probably?? ASK). Also if necessary, should we sample after or before genre summarization?

### Principal Component Analysis (PCA)

## Save Preprocessing

In [16]:
save_step(df, "step2_preprocessing")

Saved step2_preprocessing.csv
